In [34]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import matplotlib.animation as animation

#import holoviews as hv
#hv.notebook_extension()

#%output holomap='scrubber', max_frames = 10000

In [35]:
L = 250
N = 100
x = np.arange(-int(L/2), int(L/2), 1)
y = np.arange(-int(L/2), int(L/2), 1)
delta_theta = np.zeros(N)
v = 0.3

##x represents coordinates, v has the components of the velocity (probably superfluous, psi is the orientation, omega is the 
##angular velocity and r is the radius.
particles_x = np.zeros([N, 2])
particles_v = np.zeros([N, 2])
particles_psi = np.zeros(N)
particles_r = np.zeros(N)

In [36]:
for i in range (int(np.sqrt(N))):
    particles_x[10*i:10*(i+1), 0] = np.arange(0, 10)
    particles_x[10*i:10*(i+1), 1] = i
    
particles_r[:] = np.random.normal(1, 0.1, N)
particles_omega = v/particles_r[:]
particles_psi = particles_psi - delta_theta
particles_v[:, 0] = v*np.cos(particles_psi)
particles_v[:, 0] = v*np.sin(particles_psi)

In [37]:
##need to declare initial particle_omega

def calcDTheta (N, particles_x, selectMatrix):
    
    particles_dist = particles_x[None, :, :] - particles_x[:, None, :]
    
    ##we need to avoid dividing by zero
    np.fill_diagonal(particles_dist[:, :, 0], 100)
    np.fill_diagonal(particles_dist[:, :, 1], 100)
    
    angles = selectMatrix*np.arctan2(particles_dist[:, :, 1], particles_dist[:, :, 0])
    np.fill_diagonal(angles[:, :], 0)
    
    return angles
    
def update(N, L, particles_x, particles_r, particles_psi):
    F_x = np.zeros(N)
    F_y = np.zeros(N)
    
    dx = np.zeros(N)
    dy = np.zeros(N)
    
    v = 0.3
    
    theta_out = np.zeros(N)
    theta_in = np.zeros(N)
    d_theta = np.zeros(N)
    
    psi_diff = np.zeros([N, N])
    psi_diff = particles_psi[None, :] - particles_psi[:, None]
    
    ##distances calculation
    d = np.linalg.norm(particles_x[None, :, :] - particles_x[:, None, :], axis = 2)
    
    selectMatrix = np.copy(d)
    selectMatrix[selectMatrix > 2.7] = 0
    selectMatrix[selectMatrix > 0] = 1 ##the selection matrix is used to only take into account the neighbours with d < 2.7
    
    psi_diff = psi_diff * selectMatrix
    
    angles = calcDTheta(N, particles_x, selectMatrix)
    
    ##NEEDS OPTIMIZATION
    for i in range (N):
        M = max(angles[i, :])
        m = min(angles[i, :])
        if M > 0 and m < 0:
            if M - m < np.pi:
                theta_in[i] = (M-m)/2
                theta_out[:] = 2*np.pi - 2*theta_in
                d_theta = particles_psi - theta_in
        elif (M > 0 and m > 0) or (M < 0 and m < 0):
            theta_in[i] = (M-m)/2
            theta_out[:] = 2*np.pi - 2*theta_in
            d_theta = particles_psi - theta_in
    
    ##OPTIMIZATION NEEDED
    for i in range (0, N):
        for j in range (0, N):
            d[i, j] -= particles_r[i] + particles_r[j]
    d[d<0] = 0
    d = - d
    
    ##force calculations
    F = np.zeros(N)
    F[:] = v + (theta_out[:] - np.pi)*0.3*np.piecewise(theta_out[:] -np.pi, [theta_out[:] -np.pi > 0], [0, 1]) - 1*np.sum(d[:], axis =1)
    F_x[:] = F*np.cos(particles_psi[:])
    F_y[:] = F*np.sin(particles_psi[:])
    

    T = particles_omega[:]*np.piecewise(theta_out[:] -np.pi, [theta_out[:] -np.pi > 0], [0, 1]) + 1*np.random.uniform(-1,1, N) + 1*np.sum(psi_diff, axis = 1)[:]
    
    particles_x[:, 0] = np.mod(particles_x[:, 0] + F_x[:], L)
    particles_x[:, 1] = np.mod(particles_x[:, 1] + F_y[:], L)
    particles_omega[:] = - (particles_psi[:] - T*1)
    particles_psi[:] = particles_psi[:] + particles_omega[:]
    
    return particles_x, particles_psi, particles_omega

In [38]:
plt.scatter(particles_x[:, 0], particles_x[:, 1], s = np.pi*particles_r[:]**2)
plt.show()

In [43]:
def update_plot(i, fig, scat, particles_x):
    particles_temp = particles_x
    particles_x, _, _ = update(N, L, particles_x, particles_r, particles_psi)
    # Update the scatter collection, with the new colors, sizes and positions.
    scat.set_offsets(particles_x[:, :] - particles_temp[:, :])
    return particles_x

fig = plt.figure()
ax.grid(True)
ax.set_xlim([0, L])
ax.set_ylim([0, L])
scat = plt.scatter(particles_x[:, 0], particles_x[:, 1], s=np.pi*particles_r**2)
scat.set_alpha(0.8)
anim = animation.FuncAnimation(fig, update_plot(i, fig, scat, particles_x), frames = 100, interval = 100)
plt.show()

TypeError: 'numpy.ndarray' object is not callable

In [9]:
def animUpdate(i, particles_x):
    current_index = i 
    # Pick a new position for oldest rain drop, resetting its size,
    # color and growth factor.
    particles_x = update(N, L, particles_x, particles_r, particles_psi)[0]

    # Update the scatter collection, with the new colors, sizes and positions.
    scat.set_offsets(particles_x)
    return particles_x

In [13]:
animation = FuncAnimation(fig, animUpdate(i, particles_x), interval=100)
plt.show()